# Setup

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import itertools as it

import helpers_04

%matplotlib inline

# Investigating Activation Functions

## Sigmoid (Logistic) Function

$$
\sigma(x) = \frac{1}{1 + e^{-x}}
$$

$\sigma$ ranges from (0, 1). When the input $x$ is negative, $\sigma$ is close to 0. When $x$ is positive, $\sigma$ is close to 1. At $x=0$, $\sigma=0.5$

In [ ]:
def show_me(xs, ys, ylim, cross):
    fig = plt.figure(figsize=(6,4))

    plt.grid(True, which='both')
    plt.axhline(y=0, color='y')
    plt.axvline(x=0, color='y')
    
    plt.plot(xs, ys)
    plt.plot(0,cross,'ro')

    plt.ylim(ylim)

In [ ]:
xs = np.linspace(-10.0, 10.0, num=100)
sigmoid = tf.nn.sigmoid(xs)
ys = tf.Session().run(sigmoid)
show_me(xs, ys, (-0.1, 1.15), .5)

Pros:

* Easy derivative
* Function looks like we think a neuron might function: it is either off or outputing a value (up to a maximum)

Cons: 

* Not symmetric, which causes issues when training
* Susceptible to vanishing gradients: when input values are saturated (either positively or negatively), the derivative is close to zero.

##### Derivative

Derivative of the sigmoid is easy to calculate if you know the output:

$$
\frac{d\sigma}{dx} = \sigma \left(1 - \sigma \right)
$$

## Hyperbolic Tangent (Tanh)

$$
tanh(x) = \frac{sinh(x)}{cosh(x)} = \frac{e^{x} - e^{-x}}{e^{x} + e^{-x}} = \frac{e^{2x} - 1}{e^{2x} + 1}
$$

Pros:
* Similar to sigmoid, but "stretched" to range from (-1, 1)
* Symmetric around 0, which helps for optimization

Cons:
* Still suffers from vanishing gradient

In [ ]:
xs = np.linspace(-10.0, 10.0, num=100)
tanh = tf.nn.tanh(xs)
ys = tf.Session().run(tanh)
show_me(xs, ys, (-1.1, 1.1), 0.0)

##### Derivative

$$
\frac{dtanh}{dx} = 1 - tanh^{2} 
$$

## Rectified Linear Unit (ReLU)

$$
ReLU(x) = max\left(0,x\right) \\ \\
$$

Equivalent to:

$$
\begin{align*}
  ReLU(x) = \begin{cases}
    0 & \text{if $x\lt0$} \\
    x & \text{if $x\geq0$}
  \end{cases}
\end{align*}
$$

Pros:
* Incredibly easy to calculate output and derivative
* Doesn't suffer from vanishing gradient on positive side
* In practice tend to be more useful than Sigmoid/Tanh for typical activation functions

Cons:
* Not symmetric
* Can cause exploding activations if not careful
* Gradient can "die" if not careful

In [ ]:
xs = np.linspace(-10.0, 10.0, num=100)
relu = tf.nn.relu(xs)
ys = tf.Session().run(relu)
show_me(xs, ys, (-1.0, 10.0), 0.0)

##### Derivative

$$
\begin{align*}
  \frac{dReLU}{dx} = \begin{cases}
    0 & \text{if $x\lt0$} \\
    1 & \text{if $x\geq0$}
  \end{cases}
\end{align*}
$$

## Leaky ReLU

$$
LReLU(x) = max\left(\alpha x,x\right) \\ \\
$$

Equivalent to:

$$
\begin{align*}
  ReLU(x) = \begin{cases}
    \alpha x & \text{if $x\lt0$} \\
    x & \text{if $x\geq0$}
  \end{cases}
\end{align*}
$$

Pros:
* Similar to ReLU, but doesn't "die".

Cons:
* Yet another hyper-parameter to tune.

In [ ]:
def leaky_relu(z, scale):
    return tf.maximum(tf.multiply(scale, z), z)
    
xs = np.linspace(-10.0, 10.0, num=100)
l_relu = leaky_relu(xs, 0.1)
ys = tf.Session().run(l_relu)
show_me(xs, ys, (-1.0, 10.0), 0.0)
plt.annotate(r'scale ($\alpha$) set to 0.1', 
             (-5, -.5), # where the arrow points to
             (-8, 1),   # where the text lives
             size=10, 
             arrowprops={'arrowstyle': '->'});

##### Derivative

$$
\begin{align*}
  \frac{dReLU}{dx} = \begin{cases}
    \alpha & \text{if $x\lt0$} \\
    1 & \text{if $x\geq0$}
  \end{cases}
\end{align*}
$$


# Shuffling and Batching

We're going to take a very short digression to look at two utility topics.
 * We want to be able to randomly shuffle arrays in lock step (that is, apply the same permutation to several different arrays)
 * We want a method that will gives us blocks (subsets) of a dataset

In [ ]:
def shuffle(*args):
    "Shuffles list of NumPy arrays in unison"
    state = np.random.get_state()
    for array in args:
        np.random.set_state(state)
        np.random.shuffle(array)

def grouper(iter_, n):
    """Collect data into fixed-length chunks or blocks
     grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx
     from python itertools docs"""
    args = [iter(iter_)] * n
    return zip(*args)

def batches(data, labels, batch_size, randomize=True):
    if len(data) != len(labels):
        raise ValueError('Image data and label data must be same size')
    if batch_size > len(data):
        raise ValueError('Batch size cannot be larger than size of datasets')
    if randomize: 
        shuffle(data, labels)
    for res in zip(grouper(data, batch_size),
                   grouper(labels, batch_size)):
        yield res

for b in batches(list(range(10)), 
                     list(range(100,110)), 
                     3, randomize=False):
    print(b)

# Dropout

Fortunately, dropout is included in TensorFlow in the form of [`tf.nn.dropout()`](https://www.tensorflow.org/api_docs/python/tf/nn/dropout). You simply pass a `Tensor` and a desired keep probability (usually from a placeholder) into it, and it returns a `Tensor` that will have values randomly dropped at a rate of $p = 1 - keep\_prob$.  TF automatically performs the scaling necessary to keep the expected value of the outputs consistent, so you just have to pop it into your model to take advantage of it!

An interesting feature of this scaling is that it actually takes the opposite approach of what is described in the dropout academic paper. Instead of scaling the values of a neuron down _after_ training, it scales them up _during_ training. Let's take a look at the scaling so we can see what's going on under the hood.

In [ ]:
a = np.ones(10)
dropper = tf.nn.dropout(a, keep_prob=0.5) # named arg for clarity
tf.Session().run(dropper)

With a `keep_prob` of 0.5, the expected value of the inputs _without_ dropout is `2*a`. That's why we see the input values of 1 multiplied by 2.

# Creating layers with custom layer functions

You may have noticed that we end up copy-pasting a lot of code when creating neural networks. As with anything in programming, if you find yourself copying and pasting pieces of boilerplate code, it's often best to create a function that can fix the problem. Let's take a look at these three example layers:

```python
with tf.name_scope('hidden1'):
    w = tf.Variable(tf.truncated_normal([100, 300]), name='W')
    b = tf.Variable(tf.zeros([300]), name='b')
    z = tf.matmul(x, w) + b
    a = tf.nn.sigmoid(z)
 
with tf.name_scope('hidden2'):
    w = tf.Variable(tf.truncated_normal([300, 300]), name='W')
    b = tf.Variable(tf.zeros([300]), name='b')
    z = tf.matmul(x, w) + b
    a = tf.nn.sigmoid(z)
    
with tf.name_scope('output'):
    w = tf.Variable(tf.truncated_normal([300, 50]), name='W')
    b = tf.Variable(tf.zeros([50]), name='b')
    z = tf.matmul(a, w) + b
```

## Fully Connected Layers

Looking at the above, we can start breaking down the patterns we see:

* We always have weight and bias Variables, `w`, and `b`
* We the first dimension of the weight variable should be the previous layer's depth
* The depth of the layer can be any scalar
* The output layer doesn't have an activation function
* We'd like to group everything with a name scope.

Additionally, there are some other features that we'd like to look out for:

* Optionally use dropout
* Want to be able to use any activation function we want
* Want to be able to set the `w` standard deviation and `b` starting number

Here's an example layer that achieves this:

In [ ]:
def fully_connected_layer(incoming_layer, num_nodes, 
                          activation_fn=tf.nn.sigmoid, w_stddev=0.5, 
                          b_val=0.0, 
                          keep_prob=None, name=None):
    incoming_layer = tf.convert_to_tensor(incoming_layer)
    prev_num_nodes = incoming_layer.shape.dims[-1].value
    
    with tf.name_scope(name, 'fully_connected'):
        tn = tf.truncated_normal([prev_num_nodes, num_nodes], 
                                 stddev=w_stddev)
        W = tf.Variable(tn, name='W')
        const = tf.constant(b_val, shape=[num_nodes])
        b = tf.Variable(const, name='bias')
        
        z = tf.matmul(incoming_layer, W) + b
        
        # using Python's if/else expression
        # usually expect to have an activation - fallback to identity
        # we'll expect keep_prob to be None, and replace if needed
        a = activation_fn(z) if activation_fn is not None else z 
        final_a = a if keep_prob is None else tf.nn.dropout(a, keep_prob)
        
        return final_a

In [ ]:
x = tf.placeholder(tf.float32, shape=[None,20], name="x")

h1 = fully_connected_layer(x, 40, name='h1')
h2 = fully_connected_layer(h1, 20, keep_prob=.8, name='h2')

# fully connected from h2 to me (out)
out = fully_connected_layer(h2, 10, name='out')

print(h1)
print(out)

`Tensor`s have a [`TensorShape`](https://www.tensorflow.org/api_docs/python/tf/TensorShape) attribute that stores their size.  We use `incoming_layer.shape.dims[-1].value` to get the ouput size of the prior layer as a Python scalar.

The way we use this is pretty straightforward. Here's how we would replicate the previous code that had a distinct cut-and-paste feel:

    fc = fully_connected_layer(x, 300,  name='hidden1')
    fc = fully_connected_layer(fc, 300, name='hidden2')
    z  = fully_connected_layer(fc, 50,  
                               activation_fn=None, name='output')

Much more concise!

## Flattening Layers


The below code is another commonly used type of layer. It "flattens" a tensor (excluding the first dimension, which is the number of batches). For example, a matrix of ten 28x28 RGB images normally has a shape like this: `[10, 28, 28, 3]`. However, our model expects inputs to only be dimension 2. By flattening it, we "string out" the input pixels into a vector of length `28*28*3=2352`, so our final output shape is `[10, 2352]`

In [ ]:
def flatten(incoming, name=None):
    flat_shape = [-1, np.prod(incoming.shape[1:]).value]
    return tf.reshape(incoming, flat_shape)

In [ ]:
x = tf.placeholder(tf.uint8, shape=[None, 28, 28, 3], name="x")
flatten(x)

# The MNIST Dataset

##### Download and Open MNIST data

We've defined some helper functions to pull down the MNIST data.  It's sort of a pain (and this isn't the prettiest code), but it works and it lets us focus on TensorFlow.  Here goes:

In [ ]:
# this reads to ndarrays
(train_data, train_labels, 
 test_data, test_labels) = helpers_04.get_mnist_dataset()

In [ ]:
print(train_data.shape,
      train_data[0].shape, sep='\n')

Here's a quick look at what our primitive data looks like (literally):

In [ ]:
def show_image(img):
    imgplot = plt.imshow(np.squeeze(img), cmap='Greys_r')

# which image?  we could select at random 
# pick_me = np.random.randint(len(test_data))
pick_me = 0
show_image(test_data[pick_me])

Now, let's get down to business and create a model that can recognize these images.  We're going to make heavy use of `batches`, `flatten`, and `fully_connected_layer` from above.  You might want to scroll back up and remind yourself what they do.

In [ ]:
class MNIST_Model(object):
    def __init__(s):  # non-standard, for abbreviation
        graph = tf.Graph()
        with graph.as_default():
            with tf.name_scope('inputs'):
                s.x = tf.placeholder(tf.uint8, shape=[None, 28, 28, 1], name="x")
                s.y = tf.placeholder(tf.int32, shape=[None])

            with tf.name_scope('hyperparams'):
                s.learning_rate = tf.placeholder(tf.float32, [], 'learning_rate')
                s.momentum      = tf.placeholder(tf.float32, [], 'momentum')
   
                s.x_keep_prob = tf.placeholder(tf.float32, [], 'x_keep_prob')
                s.h_keep_prob = tf.placeholder(tf.float32, [], 'h_keep_prob')
            
            with tf.name_scope('preprocess'):
                x_flat  = flatten(s.x)
                x_float = tf.cast(x_flat, tf.float32) 
                s.x_dropped = tf.nn.dropout(x_float, s.x_keep_prob)
                s.one_hot_labels = tf.one_hot(s.y, 10)

            with tf.name_scope('model'):
                make_fc = fully_connected_layer # abbreviation
                
                s.h1  = make_fc(s.x_dropped, 1200, keep_prob=s.h_keep_prob, name='h1')                
                s.h2  = make_fc(s.h1,        1200, keep_prob=s.h_keep_prob, name='h2')
                s.h3  = make_fc(s.h2,        1200, keep_prob=s.h_keep_prob, name='h3')

                s.out = make_fc(s.h3, 10, activation_fn=None, name='out') 
            
            with tf.name_scope('loss'):
                smce = tf.nn.softmax_cross_entropy_with_logits
                s.loss = tf.reduce_mean(smce(logits=s.out, labels=s.one_hot_labels))

            with tf.name_scope('train'):
                # we're using a momentum optimizer to (hopefully)
                # make faster progress in regions where we'd prefer to take bigger
                # steps
                opt = tf.train.MomentumOptimizer(s.learning_rate, s.momentum)
                s.train = opt.minimize(s.loss)

            with tf.name_scope('global_step'):
                global_step = tf.Variable(0, trainable=False, name='global_step')
                s.inc_step = tf.assign_add(global_step, 1, name='inc_step')
            
            with tf.name_scope('prediction'):
                s.softmax = tf.nn.softmax(s.out, name="softmax")
                s.prediction = tf.cast(tf.arg_max(s.softmax, 1), tf.int32)
                s.pred_correct = tf.equal(s.y, s.prediction)
                s.pred_accuracy = tf.reduce_mean(tf.cast(s.pred_correct, tf.float32))    
            s.init = tf.global_variables_initializer()
            
        s.session = tf.Session(graph=graph)
        s.session.run(s.init)
    
    def fit(s, train_dict):
        tr_loss, step, tr_acc, _ = s.session.run([s.loss, s.inc_step, s.pred_accuracy, 
                                                  s.train], 
                                              feed_dict=train_dict)
        return tr_loss, step, tr_acc
    
    def predict(s, test_dict):
        ct_correct, preds = s.session.run([s.pred_correct, s.prediction], 
                                          feed_dict=test_dict)
        return ct_correct, preds

In [ ]:
# same as above
(train_data, train_labels, 
 test_data, test_labels) = helpers_04.get_mnist_dataset()

In [ ]:
mm = MNIST_Model()
for epoch in range(15):
    for batch_data, batch_labels in batches(train_data, train_labels, 100):
        m = min(.5 + epoch * 0.001, 0.99) # safeguard for very large # epochs
        train_dict = {mm.x : batch_data, 
                      mm.y : batch_labels, 

                      mm.learning_rate : 0.1,
                      mm.momentum      : m,

                      mm.x_keep_prob : 0.8, 
                      mm.h_keep_prob : 0.5}
        tr_loss, step, tr_acc = mm.fit(train_dict)
    info_update = "Epoch: {:2d} Step: {:5d} Loss: {:8.2f} Acc: {:5.2f}"
    print(info_update.format(epoch, step, tr_loss, tr_acc))

In [ ]:
# Get test accuracy
batch_correct_cts = []
for batch_data, batch_labels in batches(test_data, test_labels, 200):
    test_dict = {mm.x : batch_data,   mm.x_keep_prob : 1.0,
                 mm.y : batch_labels, mm.h_keep_prob : 1.0}
    correctness, curr_preds = mm.predict(test_dict)
    batch_correct_cts.append(correctness.sum())

print(sum(batch_correct_cts) / len(test_data))

In [ ]:
def test_and_show_images(data, labels):
    test_dict = {mm.x : data,   mm.x_keep_prob : 1.0,
                 mm.y : labels, mm.h_keep_prob : 1.0}
    
    correctness, curr_preds = mm.predict(test_dict)
    
    # setup axes
    fig, axes = plt.subplots(nrows=(len(data) // 3) + 1, ncols=3, figsize=(9,len(data)*1.5))
    axes = axes.flat
    for d, lbl, c, p, ax in zip(data, labels, correctness, curr_preds, axes):
        ax.imshow(np.squeeze(d), cmap='Greys_r') # FIXME: is cmap needed?
        ax.set_title("Predicted: {}\nCorrect: {}".format(p, bool(c)))
        ax.axis('off')
    for ax in axes: ax.set_visible(False)

indices = [25, 125]
test_and_show_images(test_data[indices], test_labels[indices])

In [ ]:
# Get correct and incorrect images
correctness = []
for batch_data, batch_labels in batches(test_data, test_labels, 200):
    test_dict = {mm.x : batch_data,   mm.x_keep_prob : 1.0,
                 mm.y : batch_labels, mm.h_keep_prob : 1.0}
    curr_correctness, curr_preds = mm.predict(test_dict)
    correctness.extend(curr_correctness)

incorrect = np.where(np.logical_not(np.array(correctness, dtype=np.bool)))[0]

In [ ]:
# viewing the incorrect images is very interesting!
indices = incorrect[:12]
test_and_show_images(test_data[indices], test_labels[indices])

# Exercise

In the model above, there are actually several buried "magic" constants.  Neural networks can be very sensitive to these values.  As an example, here are some magic values we used above:

|activation|w_stddev|b_val|learning_rate|momentum|train acc|epochs|test acc|
|-|-|-|-|-|
|sigmoid|.5|0.0|0.1|`min()` expr|mid 40s|11| 65%|

Note, that in a developmental accident, using that scenario and applying a higher keep_prob to the first hidden layer (`s.h1`) resulted in a slightly improved, and more stable (making progress), rate of convergence.  Many of these "minor" tweaks aren't that minor!

Now, try this:  Instead of using `sigmoid`s for your activation function of the fully connected layers, try using `tf.nn.relu` activations.  I'll give you two hints.  In addition, set `w_stddev=0.001` and  `bval=0.1`.  See how your network performs.

Now, there may have been "issues".  Try adjusting your learning rate to something smaller.  See how high a testing accuracy you can get.

## Solution

You can do the above, by replacing a few arguments in the `fully_connected_layer` function:

    activation_fn=tf.nn.relu, w_stddev=0.001, b_val=0.1,

You could also achieve this by passing specific arguments into the function calls (`make_fc`) to create the hidden layers in the `__init__` method of the model.  There is an examle of this in the CIFAR model exercise below (see the assignment to `shared_args`).

Your results might differ (because we have random initialization and batching steps), but here are some results from an `relu` based run:

|activation|w_stddev|b_val|learning_rate|momentum|train acc|epochs|test acc|
|-|-|-|-|-|
|relu|.001|0.1|0.01|`min()` expr|high 90s|3|98%|

Note, the test accuracy was after doing a full 15 epochs of training.

# Exercise

##### CIFAR 10

The [CIFAR-10 and CIFAR-100](https://www.cs.toronto.edu/~kriz/cifar.html) datasets consist of images comprising 10 or 100 different classes, respectively.  We can train and test from CIFAR in almost the exact same way as we did with MNIST.  

In [ ]:
import helpers_04
(train_data, train_labels,
            test_data, test_labels), readable_labels = helpers_04.load_cifar()

You'll want to look at the shape of the training/testing data.  In particular, it is going to come in pre-flattened.  So, you will have to adjust the shape you use with your models input `x` values (a `placeholder`).  Other than that, you can adjust the number of layers and the size of the layers, if you'd like.  Once you get a working model, you can experiment any number of different ways.

In [ ]:
train_data.shape

## Solution